In [0]:
'''
 @Author: Ritika Patidar
 @Date: 2021-02-23 12:15:10
 @Last Modified by: Ritika Patidar
 @Last Modified time: 2021-02-30 12:15:38  
 @Title : Twitter Sentiment Analysis using azure services
'''

Out[42]: '\n @Author: Ritika Patidar\n @Date: 2021-02-23 12:15:10\n @Last Modified by: Ritika Patidar\n @Last Modified time: 2021-02-30 12:15:38 \n @Title : Twitter Sentiment Analysis using azure services\n'

In [0]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import nltk
from textblob import TextBlob, Word, Blobber
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data] /root/nltk_data...
[nltk_data] Package averaged_perceptron_tagger is already up-to-
[nltk_data] date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data] Package brown is already up-to-date!
Out[56]: True

In [0]:
df = spark.read.csv("/FileStore/tables/secret-1.csv", header="true", inferSchema="true")
df.createOrReplaceTempView("secret")
credentials = spark.sql("select * from secret")
credential_value=credentials.collect()


In [0]:
def getblobsentiment(text):
  """
    Description : 
      Function define for find the polority of text
    Input :
      text : fetched data from twitter
    Return : 
      1 if polority_value > 0 or -1 polority_value < 0 else 0 polority_value = 0
  """
  try:
    blob = TextBlob(text)
    polority_value=0.00
    for sentence in blob.sentences:
      polority_value=polority_value+sentence.sentiment.polarity

    if polority_value > 0:
      return 1
    elif polority_value < 0:
      return -1
    else:
      return 0
  except Execption as error:
    print("{0} error occured".format(error))
  
async def run(text):
  """
    Description : 
      Function define for receive data from twitter api and distributed in batch format
    Input :
      text : fetched data from twitter
    Return : 
      None
  """
  try:
    producer = EventHubProducerClient.from_connection_string(conn_str=credential_value[0]['conn_str'],eventhub_name=credential_value[0]['eventhub_name'])
    async with producer:
      # Create a batch
      event_data_batch = await producer.create_batch()
      event_data_batch.add(EventData(text))
      await producer.send_batch(event_data_batch)
  except Exception as error:
    print("{0} error occured".format(error))

In [0]:

class listener(StreamListener):
  """
    Description : 
      This class is define for create stream lister 
  """
  def on_data(self, data):
    try:
      loop = asyncio.get_event_loop()

      t=json.loads(data)

      d1 = {"text":t["text"] ,"sentiment": getblobsentiment(t["text"]) ,"created_at" :t["created_at"]}
      loop.run_until_complete(run(json.dumps(d1)))
      print(d1)

      return(True)
    except Exception as error:
      print("{0} error occured".format(error))
   
  def on_error(self, status):
    print(status)

ckey=credential_value[0]['ckey']
csecret=credential_value[0]['csecret']
atoken=credential_value[0]['atoken']
asecret=credential_value[0]['asecret']

auth = OAuthHandler(ckey, csecret) 
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener()) 
twitterStream.filter(track=["corona"])



{'text': 'Finally getting my first Corona jab tomorrow 😭 https://t.co/ZlX1kq0HJ0', 'sentiment': 1, 'created_at': 'Mon May 03 10:12:48 +0000 2021'}
{'text': 'RT @holmenkollin: Unsere Medien erklären die Realität nicht nur, sondern erschaffen sie erst. Was wir von der Welt wissen, kommt durch die…', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:49 +0000 2021'}
{'text': 'jep... niet zo moeilijk om te bedenken.. maar ja.. de beste stuurlui staan dit keer echt aan wal', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:49 +0000 2021'}
{'text': 'Like Indian immunity is strongest due to ayurveda, we are people of the ground, we do yoga and asan and our immunit… https://t.co/Y0MUf8rSUQ', 'sentiment': -1, 'created_at': 'Mon May 03 10:12:49 +0000 2021'}
{'text': 'RT @Vuchur: Capitalism is a Death Cult the 9 millionth', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:50 +0000 2021'}
{'text': 'RT @KulturLebenB: ⭐️#KulturLebendigital-Tipp: Heimatland und #Identität- um diese Thematik dreht sich das Gespräch zwischen der italienisch…', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:50 +0000 2021'}
{'text': '@dograjournalist India’s Recovery Rate is very high and Mortality Rate is Low as compared to other nations which is… https://t.co/SH8WJd4RvX', 'sentiment': 1, 'created_at': 'Mon May 03 10:12:50 +0000 2021'}
{'text': 'RT @rishabh_memes: Everyday 3K people are dying in India because of Corona meanwhile twitter users want to know who the fuck they are in lo…', 'sentiment': -1, 'created_at': 'Mon May 03 10:12:50 +0000 2021'}
{'text': 'Corona-Inzidenz sinkt in Hamburg unter 100 https://t.co/rvOzegsv8d https://t.co/sTRELJfvuE', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:50 +0000 2021'}
{'text': 'RT @khushuuuuu: Dera Sacha Sauda has launched #DSSCovidHelpOnline where doctors are available for free consultation and advice regarding Co…', 'sentiment': 1, 'created_at': 'Mon May 03 10:12:50 +0000 2021'}
{'text': 'RT @joker_errorss: Corona அதாகரித்து வரும் நிலையில் இந்த வருடம் #IPL போட்டியை நிறுத்த வாய்ப்பு உள்ளதாக தெரிகிறது 😔😔 \n\nஆனா டேய் பாவம் #RCB 🤣…', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:50 +0000 2021'}
{'text': 'RT @MohobbatPaak: Saint Dr. @Gurmeetramrahim Singh Ji Insan instructed the administration of @derasachasauda to launch a website, on which…', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:51 +0000 2021'}
{'text': 'RT @JsrShrivastava: मैं Corona को गले लगा कर मोक्ष की प्राप्ति को तैयार हूं लेकिन STET2019 QUALIFIED होकर बेरोज़गारी से नहीं मरना चाहता।😭#y…', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:52 +0000 2021'}
{'text': 'corona eu não posso nem pensar em me aglomerar que já sobe o pânico \n\nvai embora vírus MALDITO', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:52 +0000 2021'}
{'text': '@tanweercwa @SanjayAzadSln Kahin corona target practice na karne lage', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:52 +0000 2021'}
{'text': 'RT @nhaerting: Totalitäre Phantasien der \u2066@ViolaPriesemann\u2069 im \u2066@FAZ_Feuilleton\u2069: „Die Test- und Quarantänepflichten müssen auch die Geimpf…', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:52 +0000 2021'}
{'text': '#cancle12thboardexam2021 \n@PMOIndia @CBSEWaleBhaiya @cbseindia29 \n@DrRPNishank', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:52 +0000 2021'}
{'text': 'RT @lifantita: Esta es la joya de la corona, nuestro horno al carbón.\nMirad qué cositas más ricas se cocinan aquí dentro. https://t.co/qdKa…', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:53 +0000 2021'}
{'text': 'RT @sona_sebin: Today Tamil nadu records 20738 new corona case \nAnd...\n153 deaths .\n\nஎன்ன யா ஸ்டாலின் \nஎன்ன நடக்குது தமிழ்நாட்ல?\n\nமக்கள் உ…', 'sentiment': 1, 'created_at': 'Mon May 03 10:12:53 +0000 2021'}
{'text': 'Reisebericht Mallorca retour – Warum kontrolliert der #Flughafen Zürich keine #Corona-Tests? https://t.co/xQz0kt4Juk', 'sentiment': 0, 'created_at': 'Mon May 03 10:12:53 +0000 2021'}
{'text': 'RT @navsekera: EXTREMELY IMPORTANT!\nअगर आपको ऐसे बच्चों की जानकारी हो

--------------------------------------------------------------------------- 
 IncompleteRead Traceback (most recent call last)
 /usr/local/lib/python3.8/http/client.py in _readinto_chunked (self, b) 
 591 if len ( mvb ) <= chunk_left : 
 --> 592 n = self . _safe_readinto ( mvb ) 
 593 self . chunk_left = chunk_left - n

 /usr/local/lib/python3.8/http/client.py in _safe_readinto (self, b) 
 621 if n < amt : 
 --> 622 raise IncompleteRead ( bytes ( b [ : n ] ) , amt - n ) 
 623 return n

 IncompleteRead : IncompleteRead(0 bytes read, 512 more expected)

During handling of the above exception, another exception occurred:

 IncompleteRead Traceback (most recent call last)
 /databricks/python/lib/python3.8/site-packages/urllib3/response.py in _error_catcher (self) 
 435 try : 
 --> 436 yield 
 437 

 /databricks/python/lib/python3.8/site-packages/urllib3/response.py in read (self, amt, decode_content, cache_content) 
 517 cache_content = False 
 --> 518 data = self . _fp . read ( amt ) if not fp_closed else b"" 
 519 if (

 /usr/local/lib/python3.8/http/client.py in read (self, amt) 
 457 b = bytearray ( amt ) 
 --> 458 n = self . readinto ( b ) 
 459 return memoryview ( b ) [ : n ] . tobytes ( ) 

 /usr/local/lib/python3.8/http/client.py in readinto (self, b) 
 491 if self . chunked : 
 --> 492 return self . _readinto_chunked ( b ) 
 493 

 /usr/local/lib/python3.8/http/client.py in _readinto_chunked (self, b) 
 602 except IncompleteRead : 
 --> 603 raise IncompleteRead ( bytes ( b [ 0 : total_bytes ] ) ) 
 604 

 IncompleteRead : IncompleteRead(0 bytes read)

During handling of the above exception, another exception occurred:

 ProtocolError Traceback (most recent call last)
 <command-3724289048029364> in <module> 
 30 
 31 twitterStream = Stream ( auth , listener ( ) ) 
 ---> 32 twitterStream . filter ( track = [ "corona" ] ) 
 33 

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in filter (self, follow, track, is_async, locations, stall_warnings, languages, encoding, filter_level) 
 472 self . body [ 'filter_level' ] = filter_level . encode ( encoding ) 
 473 self . session . params = { 'delimited' : 'length' } 
 --> 474 self . _start ( is_async ) 
 475 
 476 def sitestream(self, follow, stall_warnings=False,

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in _start (self, is_async) 
 387 self . _thread . start ( ) 
 388 else : 
 --> 389 self . _run ( ) 
 390 
 391 def on_closed ( self , resp ) : 

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in _run (self) 
 318 # call a handler first so that the exception can be logged. 
 319 self . listener . on_exception ( exc_info [ 1 ] ) 
 --> 320 six . reraise ( * exc_info ) 
 321 
 322 def _data ( self , data ) : 

 /usr/local/lib/python3.8/site-packages/six.py in reraise (tp, value, tb) 
 701 if value . __traceback__ is not tb : 
 702 raise value . with_traceback ( tb ) 
 --> 703 raise value
 704 finally : 
 705 value = None 

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in _run (self) 
 287 self . snooze_time = self . snooze_time_step
 288 self . listener . on_connect ( ) 
 --> 289 self . _read_loop ( resp ) 
 290 except ( Timeout , ssl . SSLError ) as exc : 
 291 # This is still necessary, as a SSLError can actually be 

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in _read_loop (self, resp) 
 337 length = 0 
 338 while not resp . raw . closed : 
 --> 339 line = buf . read_line ( ) 
 340 stripped_line = line . strip ( ) if line else line # line is sometimes None so we need to check here 
 341 if not stripped_line : 

 /databricks/python/lib/python3.8/site-packages/tweepy/streaming.py in read_line (self, sep) 
 198 else : 
 199 start = len ( self . _buffer ) 
 --> 200 self . _buffer += self . _stream . read ( self . _chunk_size ) 
 201 return six . b ( '' ) 
 202 

 /databricks/python/lib/python3.8/site-packages/urllib3/response.py in read (self, amt, decode_content, cache_con